In [8]:
import tkinter as tk
from tkinter import messagebox, simpledialog
import requests
import pandas as pd

# Define the Stock class
class Stock:
    def __init__(self, symbol, purchase_price, quantity):
        self.symbol = symbol
        self.purchase_price = purchase_price
        self.quantity = quantity
        self.current_price = purchase_price  # Initialize with purchase price

    def update_price(self, new_price):
        self.current_price = new_price

    def get_value(self):
        return self.current_price * self.quantity

    def get_profit_loss(self):
        return (self.current_price - self.purchase_price) * self.quantity

# Define the Portfolio class
class Portfolio:
    def __init__(self):
        self.stocks = []

    def add_stock(self, stock):
        self.stocks.append(stock)

    def remove_stock(self, symbol):
        self.stocks = [stock for stock in self.stocks if stock.symbol != symbol]

    def get_total_value(self):
        return sum(stock.get_value() for stock in self.stocks)

    def get_total_profit_loss(self):
        return sum(stock.get_profit_loss() for stock in self.stocks)

    def update_prices(self, api_wrapper):
        updated_prices = []
        for stock in self.stocks:
            price = api_wrapper.get_stock_quote(stock.symbol)
            if price is not None:
                stock.update_price(price)
                updated_prices.append((stock.symbol, price))
        return updated_prices

    def get_total_investment(self):
        return sum(stock.purchase_price * stock.quantity for stock in self.stocks)

    def get_total_percentage_change(self):
        total_investment = self.get_total_investment()
        if total_investment == 0:
            return 0
        return (self.get_total_profit_loss() / total_investment) * 100

    def display_performance_metrics(self):
        total_value = self.get_total_value()
        total_investment = self.get_total_investment()
        total_profit_loss = self.get_total_profit_loss()
        percentage_change = self.get_total_percentage_change()
        return (total_investment, total_value, total_profit_loss, percentage_change)

# Define the API wrapper class
class AlphaVantageAPI:
    def __init__(self, api_key):
        self.api_key = api_key
        self.base_url = "https://www.alphavantage.co/query"

    def get_stock_quote(self, symbol):
        params = {
            'function': 'GLOBAL_QUOTE',
            'symbol': symbol,
            'apikey': self.api_key
        }
        try:
            response = requests.get(self.base_url, params=params)
            response.raise_for_status()  # Check for HTTP errors
            data = response.json()
            if 'Global Quote' in data and '05. price' in data['Global Quote']:
                price = float(data['Global Quote']['05. price'])
                return price
            else:
                raise ValueError("Invalid response from Alpha Vantage API")
        except (requests.RequestException, ValueError) as e:
            print(f"Error fetching data from Alpha Vantage: {e}")
            messagebox.showerror("API Error", "Error fetching data from Alpha Vantage.")
            return None

# Define the GUI class
class StockPortfolioGUI:
    def __init__(self, root, api_wrapper):
        self.root = root
        self.root.title("Stock Portfolio Tracker")
        self.portfolio = Portfolio()
        self.api_wrapper = api_wrapper
        self.create_widgets()

    def create_widgets(self):
        # Create a frame for the buttons
        frame = tk.Frame(self.root)
        frame.pack(pady=10)

        # Add Stock Button
        self.add_button = tk.Button(frame, text="Add Stock", command=self.add_stock)
        self.add_button.grid(row=0, column=0, padx=5)

        # Remove Stock Button
        self.remove_button = tk.Button(frame, text="Remove Stock", command=self.remove_stock)
        self.remove_button.grid(row=0, column=1, padx=5)

        # View Portfolio Button
        self.view_button = tk.Button(frame, text="View Portfolio", command=self.view_portfolio)
        self.view_button.grid(row=0, column=2, padx=5)

        # Performance Metrics Button
        self.performance_button = tk.Button(frame, text="View Performance Metrics", command=self.view_performance_metrics)
        self.performance_button.grid(row=0, column=3, padx=5)

        # Update Prices Button
        self.update_prices_button = tk.Button(frame, text="Update Prices", command=self.update_prices)
        self.update_prices_button.grid(row=0, column=4, padx=5)

        # Text Area for displaying portfolio and metrics
        self.text_area = tk.Text(self.root, height=15, width=80)
        self.text_area.pack(pady=10)

    def add_stock(self):
        symbol = simpledialog.askstring("Input", "Enter stock symbol:")
        purchase_price = simpledialog.askfloat("Input", "Enter purchase price:")
        quantity = simpledialog.askinteger("Input", "Enter quantity:")
        if symbol and purchase_price and quantity:
            stock = Stock(symbol, purchase_price, quantity)
            self.portfolio.add_stock(stock)
            self.text_area.insert(tk.END, f"Added stock: {symbol} at ${purchase_price} with quantity {quantity}\n")
        else:
            messagebox.showerror("Error", "All fields must be filled out.")

    def remove_stock(self):
        symbol = simpledialog.askstring("Input", "Enter stock symbol to remove:")
        if symbol:
            self.portfolio.remove_stock(symbol)
            self.text_area.insert(tk.END, f"Removed stock: {symbol}\n")
        else:
            messagebox.showerror("Error", "Symbol must be provided.")

    def view_portfolio(self):
        self.text_area.delete(1.0, tk.END)
        if not self.portfolio.stocks:
            self.text_area.insert(tk.END, "No stocks in the portfolio.\n")
        else:
            for stock in self.portfolio.stocks:
                self.text_area.insert(tk.END, f"Symbol: {stock.symbol}, Purchase Price: ${stock.purchase_price}, Quantity: {stock.quantity}, Current Price: ${stock.current_price}\n")

    def view_performance_metrics(self):
        self.text_area.delete(1.0, tk.END)
        total_investment, total_value, total_profit_loss, percentage_change = self.portfolio.display_performance_metrics()
        self.text_area.insert(tk.END, f"Total Investment: ${total_investment:.2f}\n")
        self.text_area.insert(tk.END, f"Total Value: ${total_value:.2f}\n")
        self.text_area.insert(tk.END, f"Total Profit/Loss: ${total_profit_loss:.2f}\n")
        self.text_area.insert(tk.END, f"Total Percentage Change: {percentage_change:.2f}%\n")

    def update_prices(self):
        updated_prices = self.portfolio.update_prices(self.api_wrapper)
        self.text_area.insert(tk.END, "Updated stock prices successfully.\n")
        for symbol, price in updated_prices:
            print(f"Stock {symbol}: New Price: ${price}")  # Print updated prices to the console

if __name__ == "__main__":
    api_key = '78LM3BOULTC9YR1G'
    api_wrapper = AlphaVantageAPI(api_key)

    root = tk.Tk()
    app = StockPortfolioGUI(root, api_wrapper)
    root.mainloop()


Error fetching data from Alpha Vantage: Invalid response from Alpha Vantage API
Stock btc: New Price: $92.1419
Error fetching data from Alpha Vantage: Invalid response from Alpha Vantage API
Stock btc: New Price: $92.1419
